In [ ]:
import re
import hashlib

class AuthenticationSystem:
    def __init__(self):
        # Assuming you have a database connection and cursor, replace with your actual database setup
        self.database_connection = database_python
        self.database_cursor = None
        self.logged_in_users = set()

    def register_user(self, username, password, email):
        # Check if the username already exists
        if self.username_exists(username):
            raise UserAlreadyExistsException("Username already taken")

        # Validate email format
        if not self.is_valid_email(email):
            raise InvalidEmailFormatException("Invalid email format")

        # Hash the password securely
        hashed_password = self.hash_password(password)

        # Store user information in the database
        self.database_cursor.execute("INSERT INTO users (username, password, email) VALUES (?, ?, ?)",
                                     (username, hashed_password, email))
        self.database_connection.commit()

    def login_user(self, username, password):
        # Check if the user is already logged in
        if username in self.logged_in_users:
            raise UserAlreadyLoggedInException("User already logged in")

        # Verify the entered username and password against the stored information in the database
        user_data = self.get_user_data(username)
        if user_data and self.verify_password(password, user_data['password']):
            # Mark the user as logged in
            self.logged_in_users.add(username)
            return "Login successful"
        else:
            raise InvalidCredentialsException("Invalid credentials")

    def update_password(self, username, old_password, new_password):
        # Check if the user is logged in
        if username not in self.logged_in_users:
            raise UserNotLoggedInException("User must be logged in to update the password")

        # Verify the entered username and old password against the stored information in the database
        user_data = self.get_user_data(username)
        if user_data and self.verify_password(old_password, user_data['password']):
            # Update the password with the new password
            hashed_new_password = self.hash_password(new_password)
            self.database_cursor.execute("UPDATE users SET password = ? WHERE username = ?",
                                         (hashed_new_password, username))
            self.database_connection.commit()
            return "Password updated successfully"
        else:
            raise InvalidCredentialsException("Invalid credentials")

    def delete_account(self, username, password):
        # Check if the user is logged in
        if username not in self.logged_in_users:
            raise UserNotLoggedInException("User must be logged in to delete the account")

        # Verify the entered username and password against the stored information in the database
        user_data = self.get_user_data(username)
        if user_data and self.verify_password(password, user_data['password']):
            # Delete the user's information from the database
            self.database_cursor.execute("DELETE FROM users WHERE username = ?", (username,))
            self.database_connection.commit()
            # Remove the user from the logged-in set
            self.logged_in_users.remove(username)
            return "Account deleted successfully"
        else:
            raise InvalidCredentialsException("Invalid credentials")

    def username_exists(self, username):
        # Replace this with your actual database query to check if the username exists
        self.database_cursor.execute("SELECT COUNT(*) FROM users WHERE username = ?", (username,))
        result = self.database_cursor.fetchone()
        return result[0] > 0

    def get_user_data(self, username):
        # Replace this with your actual database query to retrieve user data
        self.database_cursor.execute("SELECT * FROM users WHERE username = ?", (username,))
        return self.database_cursor.fetchone()

    def is_valid_email(self, email):
        # Simple email format validation using regular expression
        return bool(re.match(r"[^@]+@[^@]+\.[^@]+", email))

    def hash_password(self, password):
        # Replace this with a secure password hashing library in a real-world scenario
        return hashlib.sha256(password.encode()).hexdigest()

    def verify_password(self, input_password, stored_password):
        # Replace this with the appropriate password verification mechanism
        return self.hash_password(input_password) == stored_password


# Custom exceptions for better error handling
class UserAlreadyExistsException(Exception):
    pass

class UserAlreadyLoggedInException(Exception):
    pass

class UserNotLoggedInException(Exception):
    pass

class InvalidCredentialsException(Exception):
    pass

class InvalidEmailFormatException(Exception):
    pass
